# Data Tools Dashboard
**Various tools for data manipulation and conversion**

Created: January 2026 | KIT

In [11]:
# Import required libraries
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import os
import sys
import base64
import io
import zipfile
from datetime import datetime

# Add current directory to path
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.append(current_dir)

# Import converter modules
try:
    from iv_converter_module import process_files
    from uvvis_merger_module import process_uvvis_files
    print("✅ All modules imported successfully!")
except ImportError as e:
    print(f"⚠️ Import warning: {e}")
    print("   This is normal if running for the first time.")

✅ All modules imported successfully!


In [12]:
# IV Converter Tool
class IVConverterTool:
    """Tool for converting IV measurement files"""
    
    def __init__(self):
        self.uploaded_files = {}
        self.create_widgets()
    
    def create_widgets(self):
        """Create UI widgets"""
        # Title
        self.title = widgets.HTML(
            value="<h3>📊 IV File Converter (JV Split)</h3>"
            "<p>Convert Puri JV measurement files (_ivraw.csv) to old LTI format</p>"
        )
        
        # File uploader
        self.file_uploader = widgets.FileUpload(
            accept='.csv,.zip',
            multiple=True,
            description='Upload Files',
            layout=widgets.Layout(width='100%')
        )
        self.file_uploader.observe(self._on_upload, names='value')
        
        # File list display
        self.file_list = widgets.HTML(
            value="<i>No files uploaded yet</i>",
            layout=widgets.Layout(min_height='60px', max_height='150px', overflow='auto', padding='10px')
        )
        
        # Clear button
        self.clear_button = widgets.Button(
            description='Clear Files',
            button_style='warning',
            icon='trash',
            layout=widgets.Layout(width='150px')
        )
        self.clear_button.on_click(self._on_clear)
        
        # Convert button
        self.convert_button = widgets.Button(
            description='Convert Files',
            button_style='primary',
            icon='cog',
            layout=widgets.Layout(width='150px'),
            disabled=True
        )
        self.convert_button.on_click(self._on_convert)
        
        # Status output
        self.status_output = widgets.Output(
            layout=widgets.Layout(
                border='1px solid #ddd',
                padding='10px',
                min_height='80px',
                max_height='300px',
                overflow='auto'
            )
        )
        
        # Download link placeholder
        self.download_link = widgets.HTML(value="")
    
    def _on_upload(self, change):
        """Handle file upload"""
        if not change['new']:
            return
        
        for uploaded_file in change['new']:
            filename = uploaded_file['name']
            content = uploaded_file['content']
            
            # Handle zip files
            if filename.lower().endswith('.zip'):
                try:
                    with zipfile.ZipFile(io.BytesIO(content), 'r') as zip_ref:
                        for file_info in zip_ref.filelist:
                            if file_info.filename.lower().endswith(('_ivraw.csv', '.jv.csv', '.csv')):
                                extracted_name = os.path.basename(file_info.filename)
                                self.uploaded_files[extracted_name] = zip_ref.read(file_info)
                except Exception as e:
                    with self.status_output:
                        print(f"❌ Error reading zip file {filename}: {e}")
            
            # Handle CSV files
            elif filename.lower().endswith(('_ivraw.csv', '.jv.csv', '.csv')):
                self.uploaded_files[filename] = content
        
        self._update_file_list()
        self.convert_button.disabled = len(self.uploaded_files) == 0
    
    def _update_file_list(self):
        """Update the file list display"""
        if not self.uploaded_files:
            self.file_list.value = "<i>No files uploaded yet</i>"
            return
        
        file_items = []
        for idx, filename in enumerate(self.uploaded_files.keys(), 1):
            file_items.append(f"<li>{idx}. {filename}</li>")
        
        self.file_list.value = (
            f"<b>📁 Uploaded files ({len(self.uploaded_files)}):</b>"
            f"<ul style='margin-top: 5px;'>{''.join(file_items)}</ul>"
        )
    
    def _on_clear(self, button):
        """Clear all uploaded files"""
        self.uploaded_files = {}
        self.file_uploader.value = ()
        self._update_file_list()
        self.convert_button.disabled = True
        self.download_link.value = ""
        with self.status_output:
            clear_output()
    
    def _on_convert(self, button):
        """Convert uploaded files"""
        with self.status_output:
            clear_output(wait=True)
            print("🔄 Converting files...")
        
        try:
            from iv_converter_module import process_files
            
            # Process files
            zip_content, num_processed = process_files(self.uploaded_files)
            
            if num_processed == 0:
                with self.status_output:
                    clear_output(wait=True)
                    print("⚠️ No files were processed. Check file format.")
                return
            
            # Create download link
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"converted_jv_files_{timestamp}.zip"
            
            b64_data = base64.b64encode(zip_content).decode()
            download_html = f'''
            <div style="margin: 15px 0; padding: 15px; background: #e8f5e9; border-radius: 5px; border-left: 4px solid #4caf50;">
                <h4 style="margin-top: 0; color: #2e7d32;">✅ Conversion Complete!</h4>
                <p>Processed <b>{num_processed}</b> output files from <b>{len(self.uploaded_files)}</b> input files.</p>
                <a href="data:application/zip;base64,{b64_data}" download="{filename}" 
                   style="display: inline-block; padding: 10px 20px; background: #4caf50; color: white; 
                          text-decoration: none; border-radius: 4px; font-weight: bold;">
                    📥 Download {filename}
                </a>
            </div>
            '''
            
            self.download_link.value = download_html
            
            with self.status_output:
                clear_output(wait=True)
                print(f"✅ Successfully converted {len(self.uploaded_files)} input files")
                print(f"📦 Generated {num_processed} output files")
                print(f"⬇️ Click the download button above to get your results")
        
        except Exception as e:
            with self.status_output:
                clear_output(wait=True)
                print(f"❌ Error during conversion: {e}")
                import traceback
                traceback.print_exc()
    
    def get_widget(self):
        """Return the complete widget"""
        return widgets.VBox([
            self.title,
            widgets.HTML("<hr style='margin: 10px 0;'>"),
            widgets.HTML("<b>1️⃣ Upload Files</b><br><i>Select CSV files or a ZIP file containing CSV files</i>"),
            self.file_uploader,
            self.file_list,
            widgets.HBox([self.clear_button, self.convert_button], layout=widgets.Layout(margin='10px 0')),
            widgets.HTML("<b>2️⃣ Convert and Download</b>"),
            self.download_link,
            widgets.HTML("<b>Status:</b>"),
            self.status_output
        ], layout=widgets.Layout(
            padding='20px',
            border='2px solid #1976d2',
            border_radius='8px'
        ))


print("✅ IV Converter Tool class loaded")

✅ IV Converter Tool class loaded


In [ ]:
# UV-Vis Merger Tool
class UVVisMergerTool:
    """Tool for merging UV-Vis transmission and reflection data"""
    
    def __init__(self):
        self.transmission_files = {}
        self.reflection_files = {}
        self.manual_pairs = {}  # Store manual pairing: {trans_filename: refl_filename}
        self.rename_enabled = False
        self.custom_names = {}  # Store custom names for each pair
        self.create_widgets()
    
    def create_widgets(self):
        """Create UI widgets"""
        # Title
        self.title = widgets.HTML(
            value="<h3>🌈 UV-Vis Data Merger</h3>"
            "<p>Merge transmission and reflection spectroscopy data files</p>"
        )
        
        # Transmission dropzone
        self.trans_uploader = widgets.FileUpload(
            accept='.csv,.zip',
            multiple=True,
            description='📤 Transmission',
            layout=widgets.Layout(width='100%')
        )
        self.trans_uploader.observe(self._on_trans_upload, names='value')
        
        # Reflection dropzone
        self.refl_uploader = widgets.FileUpload(
            accept='.csv,.zip',
            multiple=True,
            description='📤 Reflection',
            layout=widgets.Layout(width='100%')
        )
        self.refl_uploader.observe(self._on_refl_upload, names='value')
        
        # File lists
        self.trans_list = widgets.HTML(
            value="<i>No transmission files</i>",
            layout=widgets.Layout(min_height='60px', max_height='150px', overflow='auto', padding='10px')
        )
        
        self.refl_list = widgets.HTML(
            value="<i>No reflection files</i>",
            layout=widgets.Layout(min_height='60px', max_height='150px', overflow='auto', padding='10px')
        )
        
        # Pairing view
        self.pairing_view = widgets.HTML(
            value="<i>Upload files to see pairings</i>",
            layout=widgets.Layout(min_height='100px', max_height='300px', overflow='auto', 
                                padding='15px', border='2px solid #ddd', border_radius='5px',
                                margin='10px 0')
        )
        
        # Rename section
        self.rename_button = widgets.ToggleButton(
            value=False,
            description='📝 Enable Rename',
            button_style='',
            tooltip='Enable custom output file naming',
            icon='edit',
            layout=widgets.Layout(width='180px')
        )
        self.rename_button.observe(self._on_rename_toggle, names='value')
        
        self.rename_section = widgets.HTML(value="")
        
        # Clear buttons
        self.clear_trans_button = widgets.Button(
            description='Clear Transmission',
            button_style='warning',
            icon='trash',
            layout=widgets.Layout(width='180px')
        )
        self.clear_trans_button.on_click(self._on_clear_trans)
        
        self.clear_refl_button = widgets.Button(
            description='Clear Reflection',
            button_style='warning',
            icon='trash',
            layout=widgets.Layout(width='180px')
        )
        self.clear_refl_button.on_click(self._on_clear_refl)
        
        # Merge button
        self.merge_button = widgets.Button(
            description='Merge Files',
            button_style='primary',
            icon='compress',
            layout=widgets.Layout(width='150px'),
            disabled=True
        )
        self.merge_button.on_click(self._on_merge)
        
        # Status output
        self.status_output = widgets.Output(
            layout=widgets.Layout(
                border='1px solid #ddd',
                padding='10px',
                min_height='80px',
                max_height='300px',
                overflow='auto'
            )
        )
        
        # Download link placeholder
        self.download_link = widgets.HTML(value="")
    
    def _on_rename_toggle(self, change):
        """Handle rename toggle"""
        self.rename_enabled = change['new']
        if self.rename_enabled:
            self.rename_button.button_style = 'success'
            self.rename_button.description = '✓ Rename Enabled'
            self._update_rename_section()
        else:
            self.rename_button.button_style = ''
            self.rename_button.description = '📝 Enable Rename'
            self.rename_section.value = ""
            self.custom_names = {}
    
    def _update_rename_section(self):
        """Update the rename section"""
        if not self.rename_enabled:
            self.rename_section.value = ""
            return
        
        pairs, _, _ = self._find_matching_pairs()
        
        if not pairs:
            self.rename_section.value = "<div style='padding:15px; background:#fff3e0; border-radius:5px; margin:10px 0;'><i>Upload and pair files first to enable renaming</i></div>"
            return
        
        # Get current date as default
        from datetime import datetime
        default_date = datetime.now().strftime("%Y%m%d")
        
        html = "<div style='padding:15px; background:#f0f4ff; border-radius:5px; margin:10px 0; border:2px solid #667eea;'>"
        html += "<h4 style='margin-top:0; color:#667eea;'>✏️ Custom File Naming</h4>"
        html += "<p style='color:#555; margin:5px 0 15px 0;'>Format: <code style='background:#fff; padding:3px 8px;'>PREFIX_NAME_DATE_FIELD1_FIELD2_FIELD3.SUFFIX</code></p>"
        
        # Global fields
        html += "<div style='background:#fff; padding:15px; border-radius:5px; margin-bottom:15px;'>"
        html += "<b>Global Settings (apply to all files):</b>"
        html += "<table style='width:100%; margin-top:10px;'>"
        html += "<tr>"
        html += "<td style='padding:5px; width:15%;'><b>Prefix:</b></td>"
        html += f"<td style='padding:5px;'><input type='text' id='rename_prefix' value='KIT' oninput='updateRenamePreview()' style='width:100%; padding:5px; border:1px solid #ccc; border-radius:3px;'></td>"
        html += "<td style='padding:5px; width:15%;'><b>Name:</b></td>"
        html += "<td style='padding:5px;'><input type='text' id='rename_name' placeholder='e.g. HaGu' oninput='updateRenamePreview()' style='width:100%; padding:5px; border:1px solid #ccc; border-radius:3px;'></td>"
        html += "</tr>"
        html += "<tr>"
        html += "<td style='padding:5px;'><b>Date:</b></td>"
        html += f"<td style='padding:5px;'><input type='text' id='rename_date' value='{default_date}' placeholder='YYYYMMDD' oninput='updateRenamePreview()' style='width:100%; padding:5px; border:1px solid #ccc; border-radius:3px;'></td>"
        html += "<td style='padding:5px;'><b>Suffix:</b></td>"
        html += "<td style='padding:5px;'><input type='text' id='rename_suffix' value='.uvvis.csv' oninput='updateRenamePreview()' style='width:100%; padding:5px; border:1px solid #ccc; border-radius:3px;'></td>"
        html += "</tr>"
        html += "</table>"
        html += "</div>"
        
        # Individual fields for each pair
        html += "<div style='background:#fff; padding:15px; border-radius:5px;'>"
        html += "<b>Individual Settings (per file pair):</b>"
        html += "<table style='width:100%; margin-top:10px; border-collapse:collapse;'>"
        html += "<tr style='background:#f5f5f5; border-bottom:2px solid #ddd;'>"
        html += "<th style='padding:8px; text-align:left;'>File Pair</th>"
        html += "<th style='padding:8px; text-align:center; width:13%;'>Field 1</th>"
        html += "<th style='padding:8px; text-align:center; width:13%;'>Field 2</th>"
        html += "<th style='padding:8px; text-align:center; width:13%;'>Field 3</th>"
        html += "<th style='padding:8px; text-align:left; width:30%;'>Preview</th>"
        html += "</tr>"
        
        for idx, (trans_file, refl_file) in enumerate(pairs.items()):
            # Extract base name from transmission file
            base_name = trans_file.rsplit('.', 1)[0] if '.' in trans_file else trans_file
            base_name = base_name.replace('_T', '').replace('_t', '').replace('T', '').replace('t', '')
            
            row_color = '#fff' if idx % 2 == 0 else '#f9f9f9'
            html += f"<tr style='background:{row_color}; border-bottom:1px solid #eee;'>"
            html += f"<td style='padding:8px;'><small><code>{trans_file}</code> + <code>{refl_file}</code></small></td>"
            html += f"<td style='padding:5px;'><input type='text' id='field1_{idx}' placeholder='Field1' data-pair-idx='{idx}' data-trans='{trans_file}' oninput='updateRenamePreview()' style='width:100%; padding:4px; border:1px solid #ccc; border-radius:3px;'></td>"
            html += f"<td style='padding:5px;'><input type='text' id='field2_{idx}' placeholder='Field2' data-pair-idx='{idx}' oninput='updateRenamePreview()' style='width:100%; padding:4px; border:1px solid #ccc; border-radius:3px;'></td>"
            html += f"<td style='padding:5px;'><input type='text' id='field3_{idx}' placeholder='Field3' data-pair-idx='{idx}' oninput='updateRenamePreview()' style='width:100%; padding:4px; border:1px solid #ccc; border-radius:3px;'></td>"
            html += f"<td style='padding:8px;'><code id='preview_{idx}' style='color:#666; font-size:11px; word-break:break-all;'>KIT_NAME_{default_date}_FIELD1_FIELD2_FIELD3.uvvis.csv</code></td>"
            html += "</tr>"
        
        html += "</table>"
        html += "<button onclick='applyRenaming()' style='margin-top:15px; padding:8px 20px; background:#4caf50; color:white; border:none; border-radius:4px; cursor:pointer; font-weight:bold;'>✓ Apply Names</button>"
        html += "</div>"
        html += "</div>"
        
        # Add JavaScript for preview and applying names
        html += """
        <script>
        function updateRenamePreview() {
            const prefix = document.getElementById('rename_prefix').value || 'KIT';
            const name = document.getElementById('rename_name').value || 'NAME';
            const date = document.getElementById('rename_date').value || 'DATE';
            const suffix = document.getElementById('rename_suffix').value || '.uvvis.csv';
            
            // Update all previews
            const rows = document.querySelectorAll('[id^="field1_"]');
            rows.forEach(input => {
                const idx = input.dataset.pairIdx;
                const field1 = document.getElementById('field1_' + idx).value || 'FIELD1';
                const field2 = document.getElementById('field2_' + idx).value || 'FIELD2';
                const field3 = document.getElementById('field3_' + idx).value || 'FIELD3';
                
                const preview = `${prefix}_${name}_${date}_${field1}_${field2}_${field3}${suffix}`;
                document.getElementById('preview_' + idx).textContent = preview;
            });
        }
        
        function applyRenaming() {
            const prefix = document.getElementById('rename_prefix').value || 'KIT';
            const name = document.getElementById('rename_name').value;
            const date = document.getElementById('rename_date').value;
            const suffix = document.getElementById('rename_suffix').value || '.uvvis.csv';
            
            if (!name || !date) {
                alert('Please fill in Name and Date fields!');
                return;
            }
            
            const renameData = {
                prefix: prefix,
                name: name,
                date: date,
                suffix: suffix,
                pairs: []
            };
            
            // Collect all pair data
            const rows = document.querySelectorAll('[id^="field1_"]');
            rows.forEach(input => {
                const idx = input.dataset.pairIdx;
                const trans = input.dataset.trans;
                renameData.pairs.push({
                    trans_file: trans,
                    field1: document.getElementById('field1_' + idx).value || '',
                    field2: document.getElementById('field2_' + idx).value || '',
                    field3: document.getElementById('field3_' + idx).value || ''
                });
            });
            
            // Send to Python
            var kernel = IPython.notebook.kernel;
            if (kernel) {
                const jsonData = JSON.stringify(renameData).replace(/'/g, "\\\\'");
                var cmd = `import json; app.uvvis_merger._apply_rename_data(json.loads('${jsonData}'))`;
                kernel.execute(cmd);
            }
        }
        
        // Initialize preview immediately
        updateRenamePreview();
        </script>
        """
        
        self.rename_section.value = html
    
    def _apply_rename_data(self, data):
        """Apply rename data from JavaScript"""
        import json
        self.custom_names = {}
        
        prefix = data['prefix']
        name = data['name']
        date = data['date']
        suffix = data['suffix']
        
        for pair in data['pairs']:
            trans_file = pair['trans_file']
            field1 = pair['field1'] or 'X'
            field2 = pair['field2'] or 'Y'
            field3 = pair['field3'] or 'Z'
            
            custom_name = f"{prefix}_{name}_{date}_{field1}_{field2}_{field3}{suffix}"
            self.custom_names[trans_file] = custom_name
        
        with self.status_output:
            clear_output()
            print(f"✅ Custom names applied for {len(self.custom_names)} file pair(s)")
            print("Ready to merge with new file names!")
    
    def _find_matching_pairs(self):
        """Find matching transmission and reflection file pairs"""
        pairs = {}
        unpaired_trans = set(self.transmission_files.keys())
        unpaired_refl = set(self.reflection_files.keys())
        
        # First, use manual pairs if they exist
        for trans_file, refl_file in self.manual_pairs.items():
            if trans_file in self.transmission_files and refl_file in self.reflection_files:
                pairs[trans_file] = refl_file
                unpaired_trans.discard(trans_file)
                unpaired_refl.discard(refl_file)
        
        # Then, auto-match remaining files
        for trans_file in list(unpaired_trans):
            # Try different matching patterns
            candidates = []
            
            # Pattern 1: Replace last character (T -> R or t -> r)
            if trans_file[-5:].lower() in ['t.csv', 't.dat']:
                candidate1 = trans_file[:-5] + trans_file[-5].replace('T', 'R').replace('t', 'r') + trans_file[-4:]
                if candidate1 in unpaired_refl:
                    candidates.append(candidate1)
            
            # Pattern 2: Replace _T with _R
            if '_T.' in trans_file or '_t.' in trans_file:
                candidate2 = trans_file.replace('_T.', '_R.').replace('_t.', '_r.')
                if candidate2 in unpaired_refl:
                    candidates.append(candidate2)
            
            # Pattern 3: Try exact match with T/R swap anywhere in name
            for refl_file in unpaired_refl:
                if trans_file.replace('T', 'R').replace('t', 'r') == refl_file:
                    candidates.append(refl_file)
            
            # Use first candidate found
            if candidates:
                matched_refl = candidates[0]
                pairs[trans_file] = matched_refl
                unpaired_trans.discard(trans_file)
                unpaired_refl.discard(matched_refl)
        
        return pairs, unpaired_trans, unpaired_refl
    
    def _update_pairing_view(self):
        """Update the pairing visualization"""
        if not self.transmission_files and not self.reflection_files:
            self.pairing_view.value = "<i>Upload files to see pairings</i>"
            return
        
        pairs, unpaired_trans, unpaired_refl = self._find_matching_pairs()
        
        html = "<div style='font-family: Arial, sans-serif;'>"
        html += "<h4 style='margin-top:0; color:#333;'>📋 File Pairings</h4>"
        
        # Show paired files
        if pairs:
            html += "<div style='margin-bottom:20px;'>"
            html += "<b style='color:#4caf50;'>✓ Matched Pairs:</b>"
            html += "<table style='width:100%; border-collapse:collapse; margin-top:10px;'>"
            html += "<tr style='background:#f5f5f5; border-bottom:2px solid #ddd;'>"
            html += "<th style='padding:8px; text-align:left;'>Transmission File</th>"
            html += "<th style='padding:8px; text-align:center; width:120px;'>Action</th>"
            html += "<th style='padding:8px; text-align:left;'>Reflection File</th>"
            html += "</tr>"
            
            for idx, (trans_file, refl_file) in enumerate(pairs.items()):
                row_color = '#fff' if idx % 2 == 0 else '#f9f9f9'
                html += f"<tr style='background:{row_color}; border-bottom:1px solid #eee;'>"
                html += f"<td style='padding:8px;'><code style='background:#e3f2fd; padding:2px 6px; border-radius:3px;'>{trans_file}</code></td>"
                html += f"<td style='padding:8px; text-align:center;'>"
                html += f"<select id='pair_{idx}' onchange='window.updatePairing(\"{trans_file}\", this.value)' "
                html += f"style='padding:4px; border-radius:3px; border:1px solid #ccc; width:100%;'>"
                
                # Add current pairing
                html += f"<option value='{refl_file}' selected>{refl_file}</option>"
                
                # Add other reflection files as options
                for other_refl in self.reflection_files.keys():
                    if other_refl != refl_file:
                        html += f"<option value='{other_refl}'>{other_refl}</option>"
                
                html += "</select></td>"
                html += f"<td style='padding:8px;'><code style='background:#fff3e0; padding:2px 6px; border-radius:3px;'>{refl_file}</code></td>"
                html += "</tr>"
            
            html += "</table></div>"
        
        # Show unpaired transmission files
        if unpaired_trans:
            html += "<div style='margin-bottom:15px;'>"
            html += "<b style='color:#ff9800;'>⚠️ Unpaired Transmission Files:</b>"
            html += "<div style='margin-top:8px;'>"
            for idx, trans_file in enumerate(unpaired_trans):
                html += f"<div style='margin:5px 0; padding:8px; background:#fff3e0; border-left:4px solid #ff9800; border-radius:3px;'>"
                html += f"<code style='margin-right:10px;'>{trans_file}</code>"
                html += f"<select id='unpaired_t_{idx}' onchange='window.updatePairing(\"{trans_file}\", this.value)' "
                html += f"style='padding:4px; border-radius:3px; border:1px solid #ccc;'>"
                html += "<option value=''>-- Select Reflection File --</option>"
                for refl_file in self.reflection_files.keys():
                    html += f"<option value='{refl_file}'>{refl_file}</option>"
                html += "</select>"
                html += "</div>"
            html += "</div></div>"
        
        # Show unpaired reflection files
        if unpaired_refl:
            html += "<div>"
            html += "<b style='color:#ff9800;'>⚠️ Unpaired Reflection Files:</b>"
            html += "<div style='margin-top:8px;'>"
            for refl_file in unpaired_refl:
                html += f"<div style='margin:5px 0; padding:8px; background:#ffebee; border-left:4px solid #f44336; border-radius:3px;'>"
                html += f"<code>{refl_file}</code> <span style='color:#666;'>(no matching transmission)</span>"
                html += "</div>"
            html += "</div></div>"
        
        if pairs:
            html += f"<p style='margin-top:15px; color:#666; font-size:13px;'>"
            html += f"<b>Total:</b> {len(pairs)} pair(s) ready to merge"
            if unpaired_trans or unpaired_refl:
                html += f" | {len(unpaired_trans)} unpaired T-files | {len(unpaired_refl)} unpaired R-files"
            html += "</p>"
        
        html += "</div>"
        
        # Add JavaScript to handle pairing updates
        html += """
        <script>
        window.updatePairing = function(transFile, reflFile) {
            // Store the pairing in Python
            var kernel = IPython.notebook.kernel;
            if (kernel) {
                var cmd = `app.uvvis_merger.manual_pairs['${transFile}'] = '${reflFile}' if '${reflFile}' else app.uvvis_merger.manual_pairs.pop('${transFile}', None); app.uvvis_merger._update_pairing_view(); app.uvvis_merger._update_rename_section()`;
                kernel.execute(cmd);
            }
        }
        </script>
        """
        
        self.pairing_view.value = html
    
    def _on_trans_upload(self, change):
        """Handle transmission file upload"""
        if not change['new']:
            return
        
        for uploaded_file in change['new']:
            filename = uploaded_file['name']
            content = uploaded_file['content']
            
            if filename.lower().endswith('.zip'):
                try:
                    with zipfile.ZipFile(io.BytesIO(content), 'r') as zip_ref:
                        for file_info in zip_ref.filelist:
                            if file_info.filename.lower().endswith(('.csv', '.dat')):
                                extracted_name = os.path.basename(file_info.filename)
                                self.transmission_files[extracted_name] = zip_ref.read(file_info)
                except Exception as e:
                    with self.status_output:
                        print(f"❌ Error reading zip file {filename}: {e}")
            
            elif filename.lower().endswith(('.csv', '.dat')):
                self.transmission_files[filename] = content
        
        self._update_file_lists()
        self._update_pairing_view()
        self._update_merge_button()
        if self.rename_enabled:
            self._update_rename_section()
    
    def _on_refl_upload(self, change):
        """Handle reflection file upload"""
        if not change['new']:
            return
        
        for uploaded_file in change['new']:
            filename = uploaded_file['name']
            content = uploaded_file['content']
            
            if filename.lower().endswith('.zip'):
                try:
                    with zipfile.ZipFile(io.BytesIO(content), 'r') as zip_ref:
                        for file_info in zip_ref.filelist:
                            if file_info.filename.lower().endswith(('.csv', '.dat')):
                                extracted_name = os.path.basename(file_info.filename)
                                self.reflection_files[extracted_name] = zip_ref.read(file_info)
                except Exception as e:
                    with self.status_output:
                        print(f"❌ Error reading zip file {filename}: {e}")
            
            elif filename.lower().endswith(('.csv', '.dat')):
                self.reflection_files[filename] = content
        
        self._update_file_lists()
        self._update_pairing_view()
        self._update_merge_button()
        if self.rename_enabled:
            self._update_rename_section()
    
    def _update_file_lists(self):
        """Update file list displays"""
        if not self.transmission_files:
            self.trans_list.value = "<i>No transmission files</i>"
        else:
            items = [f"<li>{i}. {f}</li>" for i, f in enumerate(self.transmission_files.keys(), 1)]
            self.trans_list.value = f"<b>📁 Transmission files ({len(self.transmission_files)}):</b><ul style='margin-top: 5px;'>{''.join(items)}</ul>"
        
        if not self.reflection_files:
            self.refl_list.value = "<i>No reflection files</i>"
        else:
            items = [f"<li>{i}. {f}</li>" for i, f in enumerate(self.reflection_files.keys(), 1)]
            self.refl_list.value = f"<b>📁 Reflection files ({len(self.reflection_files)}):</b><ul style='margin-top: 5px;'>{''.join(items)}</ul>"
    
    def _update_merge_button(self):
        """Update merge button state"""
        pairs, _, _ = self._find_matching_pairs()
        self.merge_button.disabled = len(pairs) == 0
    
    def _on_clear_trans(self, button):
        """Clear transmission files"""
        self.transmission_files = {}
        self.trans_uploader.value = ()
        self.manual_pairs = {}
        self.custom_names = {}
        self._update_file_lists()
        self._update_pairing_view()
        self._update_merge_button()
        self._update_rename_section()
        self.download_link.value = ""
    
    def _on_clear_refl(self, button):
        """Clear reflection files"""
        self.reflection_files = {}
        self.refl_uploader.value = ()
        self.manual_pairs = {}
        self.custom_names = {}
        self._update_file_lists()
        self._update_pairing_view()
        self._update_merge_button()
        self._update_rename_section()
        self.download_link.value = ""
    
    def _on_merge(self, button):
        """Merge transmission and reflection files"""
        with self.status_output:
            clear_output(wait=True)
            print("🔄 Merging files...")
        
        try:
            from uvvis_merger_module import process_uvvis_files
            
            # Get current pairings
            pairs, unpaired_trans, unpaired_refl = self._find_matching_pairs()
            
            if not pairs:
                with self.status_output:
                    clear_output(wait=True)
                    print("⚠️ No file pairs found. Please check file names or manually pair files.")
                return
            
            # Process files
            zip_content, num_processed, errors = process_uvvis_files(
                {k: v for k, v in self.transmission_files.items() if k in pairs},
                {pairs[k]: self.reflection_files[pairs[k]] for k in pairs.keys()}
            )
            
            # Apply custom names if enabled
            if self.rename_enabled and self.custom_names:
                # Re-create zip with custom names
                import zipfile
                import io
                
                # Extract original zip
                original_zip = zipfile.ZipFile(io.BytesIO(zip_content), 'r')
                
                # Create new zip with renamed files
                new_zip_buffer = io.BytesIO()
                with zipfile.ZipFile(new_zip_buffer, 'w', zipfile.ZIP_DEFLATED) as new_zip:
                    for original_name in original_zip.namelist():
                        # Find which transmission file this belongs to
                        matching_trans = None
                        for trans_file in pairs.keys():
                            base_trans = trans_file.rsplit('.', 1)[0].replace('_T', '').replace('_t', '').replace('T', '').replace('t', '')
                            if base_trans in original_name:
                                matching_trans = trans_file
                                break
                        
                        # Use custom name if available
                        if matching_trans and matching_trans in self.custom_names:
                            new_name = self.custom_names[matching_trans]
                        else:
                            new_name = original_name
                        
                        # Copy content with new name
                        new_zip.writestr(new_name, original_zip.read(original_name))
                
                original_zip.close()
                new_zip_buffer.seek(0)
                zip_content = new_zip_buffer.read()
            
            # Create download link
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"merged_uvvis_files_{timestamp}.zip"
            
            b64_data = base64.b64encode(zip_content).decode()
            download_html = f'''
            <div style="margin: 15px 0; padding: 15px; background: #e8f5e9; border-radius: 5px; border-left: 4px solid #4caf50;">
                <h4 style="margin-top: 0; color: #2e7d32;">✅ Merge Complete!</h4>
                <p>Successfully merged <b>{num_processed}</b> file pairs.</p>
                {'<p><b>✏️ Custom file names applied</b></p>' if self.rename_enabled and self.custom_names else ''}
                <a href="data:application/zip;base64,{b64_data}" download="{filename}" 
                   style="display: inline-block; padding: 10px 20px; background: #4caf50; color: white; 
                          text-decoration: none; border-radius: 4px; font-weight: bold;">
                    📥 Download {filename}
                </a>
            </div>
            '''
            
            self.download_link.value = download_html
            
            with self.status_output:
                clear_output(wait=True)
                print(f"✅ Successfully merged {num_processed} file pairs")
                if self.rename_enabled and self.custom_names:
                    print(f"✏️ Custom file names applied")
                if unpaired_trans:
                    print(f"\n⚠️ {len(unpaired_trans)} transmission file(s) not merged (no matching reflection)")
                if unpaired_refl:
                    print(f"⚠️ {len(unpaired_refl)} reflection file(s) not merged (no matching transmission)")
                if errors:
                    print(f"\n⚠️ {len(errors)} file(s) had errors:")
                    for error in errors:
                        print(f"  • {error}")
                print(f"\n⬇️ Click the download button above to get your results")
        
        except Exception as e:
            with self.status_output:
                clear_output(wait=True)
                print(f"❌ Error during merge: {e}")
                import traceback
                traceback.print_exc()
    
    def get_widget(self):
        """Return the complete widget"""
        return widgets.VBox([
            self.title,
            widgets.HTML("<hr style='margin: 10px 0;'>"),
            widgets.HTML("<b>1️⃣ Upload Files</b>"),
            widgets.HTML("<i>Transmission and Reflection files:</i>"),
            widgets.HBox([
                widgets.VBox([
                    widgets.HTML("<b>📤 Transmission Files (T/t)</b>"),
                    self.trans_uploader,
                    self.trans_list,
                    self.clear_trans_button
                ], layout=widgets.Layout(flex='1', border='1px solid #ccc', padding='10px', margin='5px')),
                widgets.VBox([
                    widgets.HTML("<b>📤 Reflection Files (R/r)</b>"),
                    self.refl_uploader,
                    self.refl_list,
                    self.clear_refl_button
                ], layout=widgets.Layout(flex='1', border='1px solid #ccc', padding='10px', margin='5px'))
            ]),
            widgets.HTML("<b>2️⃣ Review and Adjust Pairings</b>"),
            widgets.HTML("<i>Files are automatically matched. Use dropdowns to manually adjust pairings:</i>"),
            self.pairing_view,
            widgets.HTML("<b>3️⃣ Customize Output Names (Optional)</b>"),
            self.rename_button,
            self.rename_section,
            widgets.HTML("<b>4️⃣ Merge and Download</b>"),
            self.merge_button,
            self.download_link,
            widgets.HTML("<b>Status:</b>"),
            self.status_output
        ], layout=widgets.Layout(
            padding='20px',
            border='2px solid #ff6f00',
            border_radius='8px'
        ))


print("✅ UV-Vis Merger Tool class loaded")

✅ UV-Vis Merger Tool class loaded


In [ ]:
# Main Application
class DataToolsApp:
    """Main application controller for Data Tools Dashboard"""
    
    def __init__(self):
        self.create_tools()
        self.create_ui()
    
    def create_tools(self):
        """Initialize all tools"""
        self.iv_converter = IVConverterTool()
        self.uvvis_merger = UVVisMergerTool()
    
    def create_ui(self):
        """Create the main user interface"""
        # Header with Help button
        header = widgets.HTML(
            value="""
            <div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
                        padding: 20px; border-radius: 8px; margin-bottom: 20px; color: white; 
                        display: flex; justify-content: space-between; align-items: center;'>
                <div>
                    <h1 style='margin: 0; font-size: 28px;'>🛠️ Data Tools Dashboard</h1>
                    <p style='margin: 5px 0 0 0; font-size: 14px; opacity: 0.9;'>
                        JV data splitting and UV-Vis data merging tools
                    </p>
                </div>
                <button onclick="document.getElementById('helpModal').style.display='block'" 
                        style='padding: 10px 20px; background: rgba(255,255,255,0.2); border: 2px solid white; 
                               color: white; border-radius: 5px; cursor: pointer; font-size: 14px; font-weight: bold;
                               transition: background 0.3s;'
                        onmouseover="this.style.background='rgba(255,255,255,0.3)'"
                        onmouseout="this.style.background='rgba(255,255,255,0.2)'">
                    ℹ️ Help
                </button>
            </div>
            
            <!-- Help Modal -->
            <div id='helpModal' style='display:none; position:fixed; z-index:10000; left:0; top:0; 
                                       width:100%; height:100%; background-color:rgba(0,0,0,0.5);'>
                <div style='background-color:#fefefe; margin:5% auto; padding:30px; border:1px solid #888; 
                            border-radius:10px; width:90%; max-width:700px; max-height:80vh; overflow-y:auto;'>
                    <span onclick="document.getElementById('helpModal').style.display='none'" 
                          style='color:#aaa; float:right; font-size:28px; font-weight:bold; cursor:pointer;
                                 transition: color 0.3s;'
                          onmouseover="this.style.color='#000'"
                          onmouseout="this.style.color='#aaa'">&times;</span>
                    
                    <h2 style='color:#333; margin-top:0;'>📚 How to Use Data Tools Dashboard</h2>
                    
                    <h3 style='color:#667eea; margin-top:25px;'>📊 JV Converter Tool</h3>
                    <p style='color:#555; line-height:1.6;'>
                        Converts Puri JV measurement files from <code style='background:#f4f4f4; padding:2px 6px;'>_ivraw.csv</code> format 
                        to the old LTI format for compatibility with legacy systems.
                    </p>
                    <div style='background:#e3f2fd; padding:15px; border-radius:5px; margin:15px 0;'>
                        <b>Steps:</b>
                        <ol style='margin:10px 0;'>
                            <li>Click <b>"Upload Files"</b> and select your <code>_ivraw.csv</code> files or a ZIP archive</li>
                            <li>Review the uploaded files in the list</li>
                            <li>Click <b>"Convert Files"</b> to process the files</li>
                            <li>Download the converted ZIP file with <code>.px1.jv.csv, .px2.jv.csv</code>, etc.</li>
                        </ol>
                    </div>
                    <p style='color:#666; font-size:13px;'><b>Supported formats:</b> CSV files or ZIP archives containing multiple CSV files</p>
                    
                    <h3 style='color:#ff6f00; margin-top:25px;'>🌈 UV-Vis Merger Tool</h3>
                    <p style='color:#555; line-height:1.6;'>
                        Automatically merges transmission (T) and reflection (R) spectroscopy data files by matching them 
                        based on filename patterns and aligning wavelength ranges.
                    </p>
                    <div style='background:#fff3e0; padding:15px; border-radius:5px; margin:15px 0;'>
                        <b>Steps:</b>
                        <ol style='margin:10px 0;'>
                            <li>Upload <b>Transmission files</b> (ending with T or t)</li>
                            <li>Upload <b>Reflection files</b> (ending with R or r)</li>
                            <li>The tool automatically pairs files with matching names</li>
                            <li>Click <b>"Merge Files"</b> to combine the data</li>
                            <li>Download the merged ZIP file with combined T+R data</li>
                        </ol>
                    </div>
                    <p style='color:#666; font-size:13px;'><b>Naming convention:</b> Files are matched by replacing the last character (T/R) 
                    or suffix (_T/_R). Example: <code>sample_T.csv</code> matches <code>sample_R.csv</code></p>
                    
                    <h3 style='color:#333; margin-top:25px;'>💡 Tips</h3>
                    <ul style='color:#555; line-height:1.8;'>
                        <li>Use <b>ZIP files</b> to upload multiple files at once</li>
                        <li>Click <b>"Clear"</b> buttons to remove files and start over</li>
                        <li>Check the <b>Status</b> section for processing messages and errors</li>
                        <li>All conversions are done locally - your data is not uploaded anywhere</li>
                    </ul>
                    
                    <div style='margin-top:20px; padding:15px; background:#f5f5f5; border-left:4px solid #4caf50; border-radius:5px;'>
                        <p style='color:#2e7d32; margin:0;'><b>✅ Ready to get started?</b> Close this dialog and select a tool above!</p>
                    </div>
                </div>
            </div>
            
            <script>
                // Close modal when clicking outside of it
                window.onclick = function(event) {
                    var modal = document.getElementById('helpModal');
                    if (event.target == modal) {
                        modal.style.display = 'none';
                    }
                }
            </script>
            """
        )
        
        # Info box
        info_box = widgets.HTML(
            value="""
            <div style='background: #e3f2fd; padding: 15px; border-radius: 5px; 
                        border-left: 4px solid #2196f3; margin-bottom: 20px;'>
                <b>ℹ️ Select a tool:</b>
                <ul style='margin: 10px 0 0 20px; padding: 0;'>
                    <li><b>📊 JV Tandem split:</b> Split Puri JV measurements into LTI format</li>
                    <li><b>🌈 UV-Vis Merger:</b> Merge transmission and reflection spectroscopy data</li>
                </ul>
                <p style='margin: 10px 0 0 0; font-size: 13px; color: #666;'>
                    Click the <b>ℹ️ Help</b> button in the header for detailed instructions.
                </p>
            </div>
            """
        )
        
        # Create tabs for the two main tools
        self.tabs = widgets.Tab()
        self.tabs.children = [
            self.iv_converter.get_widget(),
            self.uvvis_merger.get_widget()
        ]
        
        self.tabs.set_title(0, '📊 JV Tandem split')
        self.tabs.set_title(1, '🌈 UV-Vis Merger')
        
        # Footer
        footer = widgets.HTML(
            value="""
            <div style='text-align: center; margin-top: 30px; padding: 15px; 
                        color: #666; font-size: 12px; border-top: 1px solid #ddd;'>
                <p style='margin: 0;'>Data Tools Dashboard | KIT | 2026</p>
                <p style='margin: 5px 0 0 0;'>For questions or issues, contact your lab administrator</p>
            </div>
            """
        )
        
        # Main container
        self.container = widgets.VBox([
            header,
            info_box,
            self.tabs,
            footer
        ], layout=widgets.Layout(
            max_width='1200px',
            margin='0 auto',
            padding='20px'
        ))
    
    def get_dashboard(self):
        """Return the complete dashboard"""
        return self.container


print("✅ DataToolsApp class loaded")

✅ DataToolsApp class loaded


In [15]:
# Launch the application
try:
    # Set document title for browser tab
    display(HTML("<script>document.title='Data Tools Dashboard'</script>"))
    
    # Try to log usage if access_token module is available
    try:
        sys.path.append(os.path.dirname(os.getcwd()))
        import access_token
        access_token.log_notebook_usage()
    except ImportError:
        pass
    
    # Create and display the app
    app = DataToolsApp()
    display(app.get_dashboard())
    
    print("\n✅ Data Tools Dashboard launched successfully!")
    print("📝 Note: This app works in both Jupyter and Voilà mode")
    
except Exception as e:
    print(f"❌ Error launching application: {e}")
    import traceback
    traceback.print_exc()


✅ Data Tools Dashboard launched successfully!
📝 Note: This app works in both Jupyter and Voilà mode
